In [1]:
# helper
import math

def rangei(low, high, step = 1):
    return range(low, high + 1, step)

def get_digit(number, n):
    return number // 10**n % 10

def digits(n):
    return int(math.log10(n))+1

## Day 1

In [2]:
def fuel(mass):
    return (mass // 3) - 2

def fuel2(mass):
    res = (mass // 3) - 2
    if res <= 0:
        return 0
    return res + fuel2(res)

f = open('day1.txt')
masses = [int(v) for v in f.read().splitlines()]

print(sum([fuel(x) for x in masses]))
print(sum([fuel2(x) for x in masses]))

3337604
5003530


## Day 2

In [3]:
import os
f = open('day2.txt')
inputs = [int(code) for code in f.read().split(",")]


def day2(codes):
    codes = codes.copy()
    idx = 0
    while idx < len(codes):
        opcode = codes[idx]
        aIdx = codes[idx + 1]
        bIdx = codes[idx + 2]
        resultIdx = codes[idx + 3]
        if opcode == 99:
            return codes[0]
        elif opcode == 1:
            codes[resultIdx] = codes[aIdx] + codes[bIdx]
        elif opcode == 2:
            codes[resultIdx] = codes[aIdx] * codes[bIdx]
        else:
            raise Exception('WTF!') 
        idx += 4
    return codes[0]
    
part1Inputs = inputs.copy()
part1Inputs[1] = 12
part1Inputs[2] = 2
r = day2(inputs)

nouns = list(range(0, 100))
verbs = list(range(0, 100))
expected_output = 19690720
part2Inputs = inputs.copy()

for n in nouns:
    for v in verbs:
        part2Inputs[1] = n
        part2Inputs[2] = v
        if (day2(part2Inputs) == expected_output):
            print(f"noun {n}, verb {v}")
            print(100 * n + v)

noun 53, verb 79
5379


## Day 3

In [4]:
f = open('day3.txt')
inputs = f.read().splitlines()
first = inputs[0].split(",")
second = inputs[1].split(",")

def build_visited(instructions):
    visited = {(0, 0)} # started at the first node
    curr = (0,0)
    for p in instructions: 
        direction = p[0]
        step = int(p[1:])
        if direction == "U":
            visited.update([(curr[0], curr[1] + i + 1) for i in range(0, step)])
            curr = (curr[0], curr[1] + step)
        if direction == "R":
            visited.update([(curr[0] + i + 1, curr[1]) for i in range(0, step)])
            curr = (curr[0] + step, curr[1])
        if direction == "D":
            visited.update([(curr[0], curr[1] - i - 1) for i in range(0, step)])
            curr = (curr[0], curr[1] - step)
        if direction == "L":
            visited.update([(curr[0] - i - 1, curr[1]) for i in range(0, step)])
            curr = (curr[0] - step, curr[1])
    return visited

def manhattan_distance(x1, y1, x2, y2):
    return abs(x1 - x2) + abs(y1 - y2)


intersections = build_visited(first) & build_visited(second)
intersections.remove((0,0)) # 0,0 doesn't count
min([manhattan_distance(0,0,i[0],i[1]) for i in intersections])

1337

In [5]:
def shortest_distance(instructions, intersection):
    curr = (0,0)
    step_taken = 0
    for p in instructions: 
        direction = p[0]
        step = int(p[1:])
        if direction == "U":
            if intersection[0] == curr[0] and (curr[1] + step) > intersection[1]:
                return step_taken + abs(curr[1] - intersection[1])
            curr = (curr[0], curr[1] + step)
        if direction == "R":
            if intersection[1] == curr[1] and (curr[0] + step) > intersection[0]:
                return step_taken + abs(curr[0] - intersection[0])
            curr = (curr[0] + step, curr[1])
        if direction == "D":
            if intersection[0] == curr[0] and (curr[1] - step) < intersection[1]:
                return step_taken + abs(curr[1] - intersection[1])
            curr = (curr[0], curr[1] - step)
        if direction == "L":
            if intersection[1] == curr[1] and (curr[0] - step) < intersection[0]:
                return step_taken + abs(curr[0] - intersection[0])
            curr = (curr[0] - step, curr[1])
        step_taken += step
    return 99999

intersections = build_visited(first) & build_visited(second)
intersections.remove((0,0))
min([shortest_distance(first, i) + shortest_distance(second, i) for i in intersections])

65356

## Day 4

In [6]:
low = 240298
high = 784956

inputs = [i for i in rangei(low, high)]

def has_same_adjacents(num):
    n = digits(num)
    for i in range(1, n):
        if get_digit(num, i) == get_digit(num, i - 1):
            return True
    return False

def never_decrease(num):
    n = digits(num)
    for i in range(n - 1, 0, -1):
        if get_digit(num, i) > get_digit(num, i - 1):
            return False
    return True

qualified_password = filter(has_same_adjacents, inputs)
qualified_password = filter(never_decrease, qualified_password)
len(list(qualified_password))

1150

In [7]:
def has_same_adjacents_not_in_larger_group(num):
    n = digits(num)
    for i in range(0, n - 1):
        if get_digit(num, i) == get_digit(num, i + 1) and \
            (i == 0 or get_digit(num, i) != get_digit(num, i - 1)) and \
            (i == 4 or get_digit(num, i) != get_digit(num, i + 2)):
                return True
    return False

qualified_password = filter(has_same_adjacents_not_in_larger_group, inputs)
qualified_password = filter(never_decrease, qualified_password)
len(list(qualified_password))

748

## Day 5

In [78]:
input_file = open('day5.txt')
lines = input_file.read().splitlines()
inputs = [[int(n) for n in l.split(",")] for l in lines]

HALT = 99
ADD = 1
MULT = 2
INPUT = 3
OUTPUT = 4
JUMP_TRUE = 5
JUMP_FALSE = 6
LESS_THAN = 7
EQUALS = 8

POSITION_MODE = 0
IMMEDIATE_MODE = 1

def get_value(mode, i, programs):
    if mode == POSITION_MODE:
        return programs[programs[i]]
    if mode == IMMEDIATE_MODE:
        return programs[i]
    return None

class IntCode:
    
    def __init__(self, programs):
        self.programs = programs.copy()
        self.pointer = 0
        self.halt = False
        
    def run(self, inputs):
        programs = self.programs
        input_pointer = 0
        output = None
        
        while programs[self.pointer] != HALT:
            instruction = programs[self.pointer]
            opcode = instruction % 100
            mode1 = (instruction // 100) % 10
            mode2 = (instruction // 1000) % 10
            if opcode == ADD:
                a = get_value(mode1, self.pointer + 1, programs)
                b = get_value(mode2, self.pointer + 2, programs)
                programs[programs[self.pointer + 3]] =  a + b
                self.pointer += 4
            elif opcode == MULT:
                a = get_value(mode1, self.pointer + 1, programs)
                b = get_value(mode2, self.pointer + 2, programs)
                programs[programs[self.pointer + 3]] =  a * b
                self.pointer += 4
            elif opcode == INPUT: 
                if input_pointer > len(inputs) - 1:
                    return output # waiting for input
                programs[programs[self.pointer + 1]] = inputs[input_pointer]
                input_pointer += 1
                self.pointer += 2
            elif opcode == OUTPUT:
                output = programs[programs[self.pointer + 1]]
                self.pointer += 2
            elif opcode == JUMP_TRUE:
                a = get_value(mode1, self.pointer + 1, programs)
                b = get_value(mode2, self.pointer + 2, programs)
                self.pointer = b if a != 0 else self.pointer + 3
            elif opcode == JUMP_FALSE:
                a = get_value(mode1, self.pointer + 1, programs)
                b = get_value(mode2, self.pointer + 2, programs)
                self.pointer = b if a == 0 else self.pointer + 3
            elif opcode == LESS_THAN:
                a = get_value(mode1, self.pointer + 1, programs)
                b = get_value(mode2, self.pointer + 2, programs)
                programs[programs[self.pointer + 3]] = 1 if a < b else 0
                self.pointer += 4
            elif opcode == EQUALS:
                a = get_value(mode1, self.pointer + 1, programs)
                b = get_value(mode2, self.pointer + 2, programs)
                programs[programs[self.pointer + 3]] = 1 if a == b else 0
                self.pointer += 4
            else:
                raise Exception(f'unsupported opcode {opcode}') 
        self.halt = True
        return output
    
comp = IntCode(inputs[0])
print(comp.run([7]))

9006673


## Day 6

In [59]:
input_file = open('day6.txt')
lines = input_file.read().splitlines()

class Star:
    def __init__(self, name):
        self.name = name
        self.orbits = set()
        self.parent = None
        
    def __eq__(self, other):
        return isinstance(other, Star) and self.name == other.name

    def __hash__(self):
        return hash(self.name)

    def __str__(self):
        return self.name
    
    def __repr__(self):
        return self.name
    
    def add_orbitter(self, o):
        self.orbits.add(o)
        o.parent = self
    
def fetch_or_create_star(star_name, universe):
    if star_name in universe:
        return universe[star_name]
    universe[star_name] = Star(star_name)
    return universe[star_name]

universe = {}

for l in lines:
    station_name, orbit_name = l.split(")")
    station_star = fetch_or_create_star(station_name, universe)
    orbit_star = fetch_or_create_star(orbit_name, universe)
    station_star.add_orbitter(orbit_star)
        
def count_orbits(s):
    if len(s.orbits) == 0:
        return 0
    return len(s.orbits) + sum([count_orbits(o) for o in s.orbits])

print("part 1 - total orbits: ")
print(sum([count_orbits(s) for s in universe.values()]))

# part 2
def path_to_root(star):
    path = []
    p = star.parent
    while p != None:
        path.append(p)
        p = p.parent
    return path

print("part 2, path SAN to YOU: ")
print(len(set(path_to_root(universe["YOU"])) ^ set(path_to_root(universe["SAN"]))))

part 1 - total orbits: 
227612
part 2, path SAN to YOU: 
454


## Day 7

In [73]:
input_file = open('day7.txt')
lines = input_file.read().splitlines()
instruction = [int(i) for i in lines[0].split(',')]

import itertools

def find_max_thrust(program, number_of_amps):
    phase_settings = itertools.permutations(range(number_of_amps))
    max_thrust = 0

    max_thrust = 0
    
    for p in phase_settings:
        amps = [IntCode(program) for i in range(number_of_amps)]
        max_thurst = 0
        output = 0
        for i in range(0, len(amps)):
            output = amps[i].run([p[i], output])
            max_thrust = max(output, max_thrust)
            
    return max_thrust

find_max_thrust(instruction, 5)

92663

In [79]:
def find_max_thurst_feedback_loop(program, number_of_amps, phase_settings):
    max_thrust = 0
    
    for p in phase_settings:
        amps = [IntCode(program) for i in range(number_of_amps)]
        output = 0
        first_phase = True
        idx = 0
        
        while not amps[-1].halt:
            if first_phase:
                output = amps[idx].run([p[idx], output])
            else:
                output = amps[idx].run([output])
            idx += 1
            if idx == number_of_amps: 
                first_phase = False
                idx = 0
            
        max_thrust = max(max_thrust, output)
    
    return max_thrust

find_max_thurst_feedback_loop(instruction, 5, list(itertools.permutations(range(5, 10))))

14365052

## Day 8

In [61]:
input_file = open('day8.txt')
lines = input_file.read().splitlines()
rawInput = lines[0]


import numpy as np
np.set_printoptions(linewidth=250)


arr = np.array([int(c) for c in lines[0]])

W = 25
H = 6

arr = arr.reshape(-1, W, H,)

summary = []

for i in range(arr.shape[0]):
    zeros = np.sum([1 for x in arr[i, :, :].flat if x == 0])
    ones = np.sum([1 for x in arr[i, :, :].flat if x == 1])
    twos = np.sum([1 for x in arr[i, :, :].flat if x == 2])
    summary.append([zeros, ones, twos])

summary.sort(key=lambda x: x[0])

summary[0][1] * summary[0][2]

1064